<a href="https://colab.research.google.com/github/SolomonAmaning/Data-Science-Projects/blob/main/Responsible_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Assignment**

Implement one among cosine similarity,distance method,mutual information to detect proxies

In [19]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
data=pd.read_csv("/content/drive/MyDrive/Responsible AI assignment/compas-scores-raw.csv")

In [21]:
data.head(5)

,Person_ID,AssessmentID,Case_ID,Agency_Text,LastName,FirstName,MiddleName,Sex_Code_Text,Ethnic_Code_Text,DateOfBirth,...,RecSupervisionLevel,RecSupervisionLevelText,Scale_ID,DisplayText,RawScore,DecileScore,ScoreText,AssessmentType,IsCompleted,IsDeleted
0,50844,57167,51950,PRETRIAL,Fisher,Kevin,NaN,Male,Caucasian,12/05/92,...,1,Low,7,Risk of Violence,-2.08,4,Low,New,1,0
1,50844,57167,51950,PRETRIAL,Fisher,Kevin,NaN,Male,Caucasian,12/05/92,...,1,Low,8,Risk of Recidivism,-1.06,2,Low,New,1,0
2,50844,57167,51950,PRETRIAL,Fisher,Kevin,NaN,Male,Caucasian,12/05/92,...,1,Low,18,Risk of Failure to Appear,15.00,1,Low,New,1,0
3,50848,57174,51956,PRETRIAL,KENDALL,KEVIN,NaN,Male,Caucasian,09/16/84,...,1,Low,7,Risk of Violence,-2.84,2,Low,New,1,0
4,50848,57174,51956,PRETRIAL,KENDALL,KEVIN,NaN,Male,Caucasian,09/16/84,...,1,Low,8,Risk of Recidivism,-1.50,1,Low,New,1,0


In [22]:
data.isnull().sum()

Person_ID                      0
AssessmentID                   0
Case_ID                        0
Agency_Text                    0
LastName                       0
FirstName                      0
MiddleName                 45195
Sex_Code_Text                  0
Ethnic_Code_Text               0
DateOfBirth                    0
ScaleSet_ID                    0
ScaleSet                       0
AssessmentReason               0
Language                       0
LegalStatus                    0
CustodyStatus                  0
MaritalStatus                  0
Screening_Date                 0
RecSupervisionLevel            0
RecSupervisionLevelText        0
Scale_ID                       0
DisplayText                    0
RawScore                       0
DecileScore                    0
ScoreText                     45
AssessmentType                 0
IsCompleted                    0
IsDeleted                      0
dtype: int64

In [23]:
data.drop(columns=['MiddleName','ScoreText'], axis=1, inplace=True)

In [24]:
data.isnull().sum()

Person_ID                  0
AssessmentID               0
Case_ID                    0
Agency_Text                0
LastName                   0
FirstName                  0
Sex_Code_Text              0
Ethnic_Code_Text           0
DateOfBirth                0
ScaleSet_ID                0
ScaleSet                   0
AssessmentReason           0
Language                   0
LegalStatus                0
CustodyStatus              0
MaritalStatus              0
Screening_Date             0
RecSupervisionLevel        0
RecSupervisionLevelText    0
Scale_ID                   0
DisplayText                0
RawScore                   0
DecileScore                0
AssessmentType             0
IsCompleted                0
IsDeleted                  0
dtype: int64

In [25]:
# checking categorical variables
cat_vars = data.select_dtypes(include='object')
cat_vars.columns

Index(['Agency_Text', 'LastName', 'FirstName', 'Sex_Code_Text',
       'Ethnic_Code_Text', 'DateOfBirth', 'ScaleSet', 'AssessmentReason',
       'Language', 'LegalStatus', 'CustodyStatus', 'MaritalStatus',
       'Screening_Date', 'RecSupervisionLevelText', 'DisplayText',
       'AssessmentType'],
      dtype='object')

In [26]:
data.shape

(60843, 26)

##Applying one hot encoding to the categorical variables

In [27]:
data_encoded = pd.get_dummies(data, columns=['Agency_Text', 'LastName', 'FirstName', 'Sex_Code_Text',
       'Ethnic_Code_Text', 'DateOfBirth', 'ScaleSet', 'AssessmentReason',
       'Language', 'LegalStatus', 'CustodyStatus', 'MaritalStatus',
       'Screening_Date', 'RecSupervisionLevelText', 'DisplayText',
       'AssessmentType'
],drop_first=True)

## Converting the categorical variables to integers

In [28]:
columns_to_convert = ['Agency_Text', 'LastName', 'FirstName','Sex_Code_Text',
       'Ethnic_Code_Text', 'DateOfBirth', 'ScaleSet', 'AssessmentReason',
       'Language', 'LegalStatus', 'CustodyStatus', 'MaritalStatus',
       'Screening_Date', 'RecSupervisionLevelText', 'DisplayText',
       'AssessmentType']

for column in columns_to_convert:
    data[column] = pd.factorize(data[column])[0]

In [29]:
data.shape

(60843, 26)

In [30]:
data.dtypes

Person_ID                    int64
AssessmentID                 int64
Case_ID                      int64
Agency_Text                  int64
LastName                     int64
FirstName                    int64
Sex_Code_Text                int64
Ethnic_Code_Text             int64
DateOfBirth                  int64
ScaleSet_ID                  int64
ScaleSet                     int64
AssessmentReason             int64
Language                     int64
LegalStatus                  int64
CustodyStatus                int64
MaritalStatus                int64
Screening_Date               int64
RecSupervisionLevel          int64
RecSupervisionLevelText      int64
Scale_ID                     int64
DisplayText                  int64
RawScore                   float64
DecileScore                  int64
AssessmentType               int64
IsCompleted                  int64
IsDeleted                    int64
dtype: object

In [31]:
data.head(1)


,Person_ID,AssessmentID,Case_ID,Agency_Text,LastName,FirstName,Sex_Code_Text,Ethnic_Code_Text,DateOfBirth,ScaleSet_ID,...,Screening_Date,RecSupervisionLevel,RecSupervisionLevelText,Scale_ID,DisplayText,RawScore,DecileScore,AssessmentType,IsCompleted,IsDeleted
0,50844,57167,51950,0,0,0,0,0,0,22,...,0,1,0,7,0,-2.08,4,0,1,0


##Standardizing the variables



In [32]:
scaler = StandardScaler()

# Fit the scaler to the data
scaler.fit(data)

# Transform the data
data_standardized = scaler.transform(data)

# Convert the standardized data back to a DataFrame
data_standardized = pd.DataFrame(data_standardized, columns=data.columns)

##Confirming standardization and observing the difference in original and standardized data

In [33]:
data.head(2)

,Person_ID,AssessmentID,Case_ID,Agency_Text,LastName,FirstName,Sex_Code_Text,Ethnic_Code_Text,DateOfBirth,ScaleSet_ID,...,Screening_Date,RecSupervisionLevel,RecSupervisionLevelText,Scale_ID,DisplayText,RawScore,DecileScore,AssessmentType,IsCompleted,IsDeleted
0,50844,57167,51950,0,0,0,0,0,0,22,...,0,1,0,7,0,-2.08,4,0,1,0
1,50844,57167,51950,0,0,0,0,0,0,22,...,0,1,0,8,1,-1.06,2,0,1,0


In [34]:
data_standardized.head(2)

,Person_ID,AssessmentID,Case_ID,Agency_Text,LastName,FirstName,Sex_Code_Text,Ethnic_Code_Text,DateOfBirth,ScaleSet_ID,...,Screening_Date,RecSupervisionLevel,RecSupervisionLevelText,Scale_ID,DisplayText,RawScore,DecileScore,AssessmentType,IsCompleted,IsDeleted
0,-0.197668,-1.488225,-0.856896,-0.66979,-1.188958,-0.981519,-0.529649,-0.970135,-1.519199,0.193505,...,-1.634664,-0.667274,-0.705161,-0.805387,-1.224745,-0.710431,0.163608,-0.289469,0.0,0.0
1,-0.197668,-1.488225,-0.856896,-0.66979,-1.188958,-0.981519,-0.529649,-0.970135,-1.519199,0.193505,...,-1.634664,-0.667274,-0.705161,-0.604040,0.000000,-0.609245,-0.600383,-0.289469,0.0,0.0


## Using sample to avoid crashing in iteration

In [35]:
n_rows = 100

# Randomly select n_rows from the dataframe
data_standardized = data_standardized.sample(n_rows)

##Using Cosine similarity to find the proxy features similar to the protected features

In [41]:

protected_features=data_standardized[['Person_ID','LastName',	'FirstName','DateOfBirth']]
# Compute the cosine similarity between all pairs of columns
similarity = pairwise_distances(data_standardized.drop(columns=protected_features.columns), metric='cosine')

# Identify the columns that are most similar to the protected features
# You can set a threshold to select the most similar features 
threshold = 0.9
proxy_columns = [data_standardized.columns[i] for i in range(len(data_standardized.columns)) if any(similarity[i] > threshold) and data_standardized.columns[i] not in protected_features.columns ]

In [42]:
print("Proxy columns for protected features: ", proxy_columns)

Proxy columns for protected features:  ['AssessmentID', 'Case_ID', 'Agency_Text', 'Sex_Code_Text', 'Ethnic_Code_Text', 'ScaleSet_ID', 'ScaleSet', 'AssessmentReason', 'Language', 'LegalStatus', 'CustodyStatus', 'MaritalStatus', 'Screening_Date', 'RecSupervisionLevel', 'RecSupervisionLevelText', 'Scale_ID', 'DisplayText', 'RawScore', 'DecileScore', 'AssessmentType', 'IsCompleted', 'IsDeleted']
